# **Predictive Default Risk Assessor V.01**

# TODO

* Base model 
* Comparison
* Specialised
* For small entities - Examples?
* Backtest - All sectors 
* Understanding the model across all sectors/industries
* Any markets - consumer goods, industries
* UI last step after backtesting

In [1]:
model_inputs = {
    "profitability": {
        "class_weight": 0.30,
        "weights": [1.0], 
        "metrics": {
            "oper_margin": {
                "lower_is_better": False,
                "thresholds": [
                    (40, float("inf")),
                    (35, 39),
                    (30, 34),
                    (25, 29),
                    (20, 24),
                    (15, 19),
                    (10, 14),
                    (5, 9),
                    (float("-inf"), 0)
                ],
            }
        },
    },
    "leverage_coverage": {
        "class_weight": 0.55,
        "weights": [0.4, 0.3, 0.3],
        "metrics": {
            "tot_debt_to_tot_eqy": {
                "lower_is_better": True,
                "thresholds": [
                     (float("-inf"), 2.0),
                     (2.0, 16.0),
                     (16.0, 24.0),
                     (24.0, 33.0),
                     (33.0, 43.0),
                     (43.0, 54.0),
                     (54.0, 68.0),
                     (68.0, 94.0),
                     (94.0, float("inf")),
                ],
            },
            "tot_debt_to_ebitda": {
                "lower_is_better": True,
                "thresholds": [
                    (float("-inf"), 0.09),
                    (0.09, 0.49),
                    (0.49, 0.9),
                    (0.9, 1.36),
                    (1.36, 1.68),
                    (1.68, 2.26),
                    (2.26, 3.27),
                    (3.27, 4.4),
                    (4.4, float("inf")),
                ],
            },
            "ebitda_to_tot_int_exp": {
                "lower_is_better": False,
                "thresholds": [
                    (25, float("inf")),
                    (20, 25),
                    (15, 20),
                    (10, 15),
                    (5, 10),
                    (3, 5),
                    (1, 3),
                    (0, 1),
                    (float("-inf"), 0),
                ],
            },
        },
    },
    "efficiency": {
        "class_weight": 0.15,
        "weights": [0.5, 0.5],
        "metrics": {
            "return_on_asset": {
                "lower_is_better": False,
                "thresholds": [
                    (0.15, float("inf")),
                    (0.10, 0.15),
                    (0.08, 0.10),
                    (0.06, 0.08),
                    (0.04, 0.06),
                    (0.02, 0.04),
                    (0.00, 0.02),
                    (-0.02, 0.00),
                    (float("-inf"), -0.02)
                ],
            },
            "asset_turnover": {
                "lower_is_better": False,
                "thresholds": [
                    (4.0, float("inf")),
                    (3.0, 4.0),
                    (2.0, 3.0),
                    (1.5, 2.0),
                    (1.0, 1.5),
                    (0.75, 1.0),
                    (0.5, 0.75),
                    (0.25, 0.5),
                    (float("-inf"), 0.25)
                ],
            },
        },
    },
}

In [2]:
import pandas as pd
import numpy as np

In [3]:
class CreditRatingCalculator:
    def __init__(self, metrics):
        self.metrics = metrics
        
    def _calculate_metric_score(self, metric, thresholds, inverse):
        for score, (lower, upper) in enumerate(thresholds, start=1):
            if (inverse and metric <= upper) or (not inverse and metric >= lower):
                return score
        return len(thresholds) // 2 # else return the middle score

    def _calculate_category_score(self, category_metrics, ratios):
        total_weighted_score = 0

        for metric, weight in zip(
            category_metrics["metrics"].items(), category_metrics["weights"]
        ):
            metric_name, metric_data = metric
            value = ratios[metric_name]
            score = self._calculate_metric_score(
                value, metric_data["thresholds"], metric_data["lower_is_better"]
            )
            total_weighted_score += score * weight

        return total_weighted_score

    def _calculate_scores(self, ratios):
        scores = {}
        for category, category_data in self.metrics.items():
            category_score = self._calculate_category_score(category_data, ratios)
            scores[category] = category_score
        return scores

    def _calculate_weighted_score(self, scores):
        weights = {
            category: category_data["class_weight"]
            for category, category_data in self.metrics.items()
        }
        return sum(scores[category] * weight for category, weight in weights.items())

    def _determine_credit_rating(self, weighted_score):
        # credit_ratings = [
        #     (1.5, "Aaa"),
        #     (2.5, "Aa"),
        #     (3.5, "A"),
        #     (4.5, "Baa"),
        #     (5.5, "Ba"),
        #     (6.5, "B"),
        #     (7.5, "Caa"),
        #     (8.5, "Ca"),
        #     (float("inf"), "C"),
        # ]
        
        credit_ratings = [
            (2.5, "Aaa"),
            (3.5, "Aa"),
            (4.5, "A"),
            (5.5, "Baa"),
            (6.5, "Ba"),
            (7.5, "B"),
            (8.5, "Caa"),
            (9.5, "Ca"),
            (float("inf"), "C"),
        ]

        for threshold, rating in credit_ratings:
            if weighted_score < threshold:
                return rating

    def calculate_credit_rating(self, ratios):
        self.scores = self._calculate_scores(ratios)
        self.credit_score = self._calculate_weighted_score(self.scores)
        self.credit_rating = self._determine_credit_rating(self.credit_score)

In [4]:
# df = pd.read_csv("research/JALSH Index_dataset_2000_2024_clean.csv", index_col=0, header=[0, 1])
# classfier = pd.read_excel("research/classification_data.xlsx", index_col=0)
metrics = pd.read_excel("research/metrics_full.xlsx", index_col=0)

In [5]:
company = "VOD SJ Equity"

In [6]:
model_metrics = [
    "oper_margin", 
    "tot_debt_to_tot_eqy", 
    "tot_debt_to_ebitda", 
    "ebitda_to_tot_int_exp", 
    "return_on_asset",
    "asset_turnover",
]

ratios = metrics[company].loc[model_metrics].to_dict()

In [7]:
model = CreditRatingCalculator(model_inputs)
model.calculate_credit_rating(ratios)
print(f"Model Inputs:")
display(ratios)
print("")
print(f"Class Scoring: {model.scores}")
print(f"Credit Score: {model.credit_score}")
print(f"Credit Rating: {model.credit_rating}")

Model Inputs:


{'oper_margin': 26.21094703807191,
 'tot_debt_to_tot_eqy': 65.00493973766227,
 'tot_debt_to_ebitda': 0.7931121155634536,
 'ebitda_to_tot_int_exp': 13.35518524144715,
 'return_on_asset': 16.14384135791923,
 'asset_turnover': 1.076701663147559}


Class Scoring: {'profitability': 4.0, 'leverage_coverage': 4.9, 'efficiency': 3.0}
Credit Score: 4.345000000000001
Credit Rating: A


In [21]:
df = pd.read_excel("resources/stock_universe_default_prob.xlsx", index_col=0)
ratings = df.sort_values(["bb_1yr_default_prob"], ascending=False)

In [34]:
ratings_dict = {}
for company in ratings.index:
    try:
        ratios = metrics[company].loc[model_metrics].to_dict()
        model = CreditRatingCalculator(model_inputs)
        model.calculate_credit_rating(ratios)
        ratings_dict[company] = {
            "model_rating": model.credit_rating,
            "model_score": model.credit_score,
            "model_class_scoreing": model.scores,
            "bloomberg_rating": ratings.loc[company]["rsk_bb_issuer_default"], 
            "bloomberg_default_prob": ratings.loc[company]['bb_1yr_default_prob']
        }
    except: pass

In [36]:
data = pd.DataFrame(ratings_dict).T

In [37]:
data.to_excel("dat.xlsx")

In [ ]:
import matplotlib.pyplot as plt

data = { 
    'TCP SJ Equity': {'Model': 'B', 'Bloomberg': 'HY6'},
    'MTA SJ Equity': {'Model': 'Baa', 'Bloomberg': 'HY3'},
    'PIK SJ Equity': {'Model': 'Ba', 'Bloomberg': 'HY3'},
    'BLU SJ Equity': {'Model': 'Baa', 'Bloomberg': 'HY2'},
    'TKG SJ Equity': {'Model': 'Ba', 'Bloomberg': 'HY2'},
    'KAP SJ Equity': {'Model': 'B', 'Bloomberg': 'HY1'},
    'HMN SJ Equity': {'Model': 'Ba', 'Bloomberg': 'HY1'},
    'SOL SJ Equity': {'Model': 'Baa', 'Bloomberg': 'HY1'}
}

model_ratings = ['Aaa', 'Aa', 'A', 'Baa', 'Ba', 'B', 'Caa', 'Ca', 'C']
bloomberg_ratings = ['IG1', 'IG2', 'IG3', 'IG4', 'IG5', 'IG6', 'IG7', 'IG8', 'IG9','IG10', 'HY1', 'HY2', 'HY3', 'HY4', 'HY5', 'HY6']

fig, ax = plt.subplots(figsize=(10, 6))

for company, ratings in data.items():
    model_rating = model_ratings.index(ratings['Model'])
    bloomberg_rating = bloomberg_ratings.index(ratings['Bloomberg'])
    ax.plot([model_rating, bloomberg_rating], [0, 1], marker='o', linestyle='-', label=company)

ax.set_yticks([0, 1])
ax.set_yticklabels(['Moodys', 'Bloomberg'])
ax.set_xticks(range(len(model_ratings + bloomberg_ratings)))
ax.set_xticklabels(model_ratings + bloomberg_ratings, rotation=45, ha='right')
ax.grid(True)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()